# Lab 4 - SQL


## Due: Thursday, Oct 4, 2018, 11:59:00pm

### Submission instructions
After completing this homework, you will turn in two files via Canvas ->  Assignments -> Lab 4:
Your Notebook, named si330-lab4-YOUR_UNIQUE_NAME.ipynb and
the HTML file, named si330-lab4-YOUR_UNIQUE_NAME.html

### Name:  Samantha Cohen 
### Uniqname: samcoh
### People you worked with: Rhea, Will, Emil 


## Objectives
After completing this Lab, you should know how to use NLTK to:
* Insert data into a database
* Query a database

## Part 0: Setup + Loading the data

In [1]:
import sqlite3 as sqlite
import xml.etree.ElementTree as ET

### Part 0.1: SQLite DB Browser

Go to http://sqlitebrowser.org/ and install the SQLite DB Browser GUI. You can use this tool to inspect an SQLite database and test SQL commands on it outside of python.

### Part 0.2: Load data

For this lab, we will be loading in some data from two XML files and then insert them into the database. **You don't need to modify the code in this section.**

In [2]:
# DON'T MODIFY THIS --- it is used for loading the XML data that we will be inserting into the database
def read_xml_file(xml_filename, field_types):
    tree = ET.parse(xml_filename)
    root = tree.getroot()
    lot = []
    for child in root:
        fields = []
        for gc in child:
            fields.append(gc.text)

        field_list = []
        count = 0
        for f in fields:
            t = field_types[count]
            if t == 'int':
                field_list.append(int(f))
            elif t == 'string':
                field_list.append(f)
            elif t == 'float':
                field_list.append(float(f))
            else:
                field_list.append(f)
            count = count + 1

        lot.append(tuple(field_list))
    return lot

Now we'll load the data into `customers`:

In [3]:
# The format of the customer.xml file records:
# <T>
#    <C_CUSTKEY>1</C_CUSTKEY>                       #in the database we will call this customer_id
#    <C_NAME>Customer#000000001</C_NAME>            #... name
#    <C_ADDRESS>IVhzIApeRb ot,c,E</C_ADDRESS>       #... address
#    <C_NATIONKEY>15</C_NATIONKEY>                  #... nation_id
#    <C_PHONE>25-989-741-2988</C_PHONE>             #... phone
#    <C_ACCTBAL>711.56</C_ACCTBAL>                  #... balance
#    <C_MKTSEGMENT>BUILDING</C_MKTSEGMENT>          #... market_segment
#    <C_COMMENT>lorem ipsum</C_COMMENT>             #... comment
# </T>
customers = read_xml_file('customer.xml',
                    ['int', 'string', 'string', 'int', 'string', 'float', 'string', 'string'])

`customers` is a list of tuples, where each tuple represents a customer. Here are the first couple of customers:

In [4]:
customers[:5]

[(1,
  'Customer#000000001',
  'IVhzIApeRb ot,c,E',
  15,
  '25-989-741-2988',
  711.56,
  'BUILDING',
  'regular, regular platelets are fluffily according to the even attainments. blithely iron'),
 (2,
  'Customer#000000002',
  'XSTf4,NCwDVaWNe6tEgvwfmRchLXak',
  13,
  '23-768-687-3665',
  121.65,
  'AUTOMOBILE',
  'furiously special deposits solve slyly. furiously even foxes wake alongside of the furiously ironic ideas. pending'),
 (3,
  'Customer#000000003',
  'MG9kdTD2WBHm',
  1,
  '11-719-748-3364',
  7498.12,
  'AUTOMOBILE',
  'special packages wake. slyly reg'),
 (4,
  'Customer#000000004',
  'XxVSJsLAGtn',
  4,
  '14-128-190-5944',
  2866.83,
  'MACHINERY',
  'slyly final accounts sublate carefully. slyly ironic asymptotes nod across the quickly regular pack'),
 (5,
  'Customer#000000005',
  'KvpyuHCplrB84WgAiGV6sYpZq7Tj',
  3,
  '13-750-942-6364',
  794.47,
  'HOUSEHOLD',
  'blithely final instructions haggle; stealthy sauternes nod; carefully regu')]

There are plenty more customers:

In [5]:
len(customers)

1500

We'll also load another dataset into the `nations` variable:

In [6]:
# The format of the nation.xml file records:
# <T>
#        <N_NATIONKEY>0</N_NATIONKEY>           #in the database we will call this nation_id
#        <N_NAME>ALGERIA</N_NAME>               #... name
#        <N_REGIONKEY>0</N_REGIONKEY>           #... region_id
#        <N_COMMENT>lorem ipsum</N_COMMENT>     #... comment
# </T>
nations = read_xml_file(r'nation.xml', ['int', 'string', 'int', 'string'])

The first few entries look like this:

In [7]:
nations[:5]

[(0, 'ALGERIA', 0, 'final accounts wake quickly. special reques'),
 (1,
  'ARGENTINA',
  1,
  'idly final instructions cajole stealthily. regular instructions wake carefully blithely express accounts. fluffi'),
 (2, 'BRAZIL', 1, 'always pending pinto beans sleep sil'),
 (3, 'CANADA', 1, 'foxes among the bold requests'),
 (4,
  'EGYPT',
  4,
  'pending accounts haggle furiously. furiously bold accounts detect. platelets at the packages haggle caref')]

In [8]:
len(nations)

25

## Part 1: Database initialization

SQLite database are just stored as files on disk, so to connect to one you can just specify the name of the file you want to write the database to, as in `sqlite.connect('si-330-lab4.db')`.

### Part 1.1 Open database connection

To open a connection to an SQLite database, use the `sqlite.connect()` function. Normally we would probably use `sqlite.connect()` paired with a `with` statement, as in:

```python
with sqlite.connect('si-330-lab6.db') as con:
    # database code here...
```

In this form, the `with` statement will automatically close the database connection at the end of the block.

**However,** this does not work as well in a notebook --- we would have to put a `with` statement around every block that uses the database. So instead, we'll open the database separately. **This means that you have to rememeber to close it later!**

To open a connection to the database:

In [9]:
con = sqlite.connect('si-330-lab4.db')

# changing the row factory allows us to access fields by name:
# rows will be returned as dictionaries instead of lists
con.row_factory = sqlite.Row

# We'll also want a "cursor" to use to execute queries:
cur = con.cursor()

If you need to close your database connection, execute this block:

In [10]:
# don't execute now ...
#con.close()

### Part 1.1: Creating and populating a table

The first thing we'll do is create and populate the `customer` table. We're going to insert all the values from the `customers` datastructure from Part 0 into this table.

To populate the table, we'll first remove it in case it already exists:

In [11]:
cur.execute("DROP TABLE IF EXISTS customer")
con.commit() # commit these changes to the database

Next, write a `CREATE TABLE` statement that creates a table with the following columns:

- customer_id (integer) - this should also be a **primary key**
- name (text)
- address (text)
- nation_id (integer)
- phone (text)
- balance (real),
- market_segment (text)
- comment (text)

In [12]:
# CHANGE ME: replace `None` with a string containing an SQL CREATE TABLE statement
statement = '''
CREATE TABLE 'customer'(
'customer_id' INTEGER PRIMARY KEY, 
'name' TEXT,
'address' TEXT,
'nation_id' INTEGER,
'phone' TEXT,
'balance' REAL, 
'market_segment' TEXT, 
'comment' TEXT)
'''
cur.execute(statement)

Next, populate the table with values from `customers` using an `INSERT INTO` statement:

In [13]:
# CHANGE ME
print(len(customers))
#print(len(nation_id))
statement = 'INSERT INTO customer VALUES(?,?,?,?,?,?,?,?)'
cur.executemany(statement,customers)
con.commit()
#for x in customers:
    #print(x)

    #cur.commit() 

1500


### Part 1.2: Adding another table

Now we're going to insert all the values from the `nations` datastructure from Part 0 into the `nation` table.

To populate the table, we'll first remove it in case it already exists:

In [14]:
cur.execute("DROP TABLE IF EXISTS nation")
con.commit() # commit these changes to the database

Next, write a `CREATE TABLE` statement that creates a table with the following columns:

- nation_id (integer) - this should also be a **primary key**
- name (text)
- region_id (integer)
- comment (text)

In [15]:
# CHANGE ME: replace `None` with a string containing an SQL CREATE TABLE statement
statement = '''
CREATE TABLE 'nation'(
'nation_id' INTEGER PRIMARY KEY, 
'name' TEXT,
'region_id' INTEGER,
'comment' TEXT)
'''
cur.execute(statement)
  

Next, populate the table with values from `nations` using an `INSERT INTO` statement:

In [16]:
# CHANGE ME
statement = "INSERT INTO nation VALUES(?,?,?,?)"
cur.executemany(statement,nations)
con.commit()

### Part 1.3: Checking your work

Open up the **SQLiteBrowser** GUI and open your database file (`"si-330-lab4.db"`). Under `Browse Data`, look at the contents of the `customer` table. It should look like this:

![SQLiteBrowser view of customer table](sqlite-browser.PNG)

For complex queries, it is often easier to debug them using the GUI than python. Click on the **Execute SQL** tab to enter SQL directly within the GUI. 

## Part 2: Customer Queries

**2.1** Write an SQL query to compute the count of all records (rows) in the customer table

In [17]:
# CHANGE ME
statement = '''
SELECT COUNT(*) as Customer_Count
FROM customer
'''
cur.execute(statement)
# this line just prints the results so you can see them...
[dict(row) for row in cur.fetchall()]

[{'Customer_Count': 1500}]

**2.2** Write an SQL query to compute the average balance over all rows in the customer table.

In [18]:
# CHANGE ME
statement = '''
SELECT AVG(balance) as Average_Balance
FROM customer
'''
cur.execute(statement)

[dict(row) for row in cur.fetchall()]

[{'Average_Balance': 4454.577060000001}]

**2.3** Write an SQL query to get the customer_id, name, and balance columns from the customer table that
have balance >= 1000, and sort the results by customer_id in descending order (largest to smallest)

In [19]:
# CHANGE ME
statement = '''
SELECT customer_id,name,balance 
FROM customer 
WHERE balance >= 1000 
ORDER BY customer_id DESC 
'''
cur.execute(statement)

# just print the top 10 (there's a lot)
[dict(row) for row in cur.fetchall()][:10]

[{'customer_id': 1500, 'name': 'Customer#000001500', 'balance': 6910.79},
 {'customer_id': 1499, 'name': 'Customer#000001499', 'balance': 9128.69},
 {'customer_id': 1498, 'name': 'Customer#000001498', 'balance': 5810.56},
 {'customer_id': 1497, 'name': 'Customer#000001497', 'balance': 2449.57},
 {'customer_id': 1495, 'name': 'Customer#000001495', 'balance': 6227.55},
 {'customer_id': 1494, 'name': 'Customer#000001494', 'balance': 8292.21},
 {'customer_id': 1493, 'name': 'Customer#000001493', 'balance': 7014.12},
 {'customer_id': 1491, 'name': 'Customer#000001491', 'balance': 3739.82},
 {'customer_id': 1490, 'name': 'Customer#000001490', 'balance': 8997.6},
 {'customer_id': 1489, 'name': 'Customer#000001489', 'balance': 4389.66}]

**2.4** Write an SQL query to sum the balance field by market_segment (call this total column total_balance)
and sort by total_balance in descending order.
The output columns should be market_segment and total_balance (the sum over balance)

In [20]:
# CHANGE ME
statement = '''
SELECT market_segment,SUM(balance) as total_balance
FROM customer
GROUP BY market_segment
ORDER BY total_balance DESC
'''
cur.execute(statement)

[dict(row) for row in cur.fetchall()]

[{'market_segment': 'BUILDING', 'total_balance': 1444587.8000000007},
 {'market_segment': 'AUTOMOBILE', 'total_balance': 1395695.720000001},
 {'market_segment': 'MACHINERY', 'total_balance': 1296958.6099999994},
 {'market_segment': 'HOUSEHOLD', 'total_balance': 1279340.6600000013},
 {'market_segment': 'FURNITURE', 'total_balance': 1265282.7999999996}]

## Part 3: Join Nation and Customer

**3.1** Write an SQL query to compute the total balance in the customer table for each nation.    The output columns should be name (country name) from the "nation" table and the total account balance, i.e. the sum of balance in the "customer" table for records grouped by country, output as a column named total_balance.    The result should be sorted alphabetically by country name.

**HINT: some columns have the same names in both tables, so you will have to refer to them specifically (e.g. "nation.name" or "customer.name" instead of just "name")**

In [21]:
# CHANGE ME
statement= '''
SELECT SUM(c.balance) as total_balance, n.name as country_name
FROM customer as c 
JOIN nation as n 
ON c.nation_id = n.nation_id
GROUP BY country_name
ORDER BY country_name
'''
cur.execute(statement)

[dict(row) for row in cur.fetchall()]

[{'total_balance': 248180.18999999997, 'country_name': 'ALGERIA'},
 {'total_balance': 286203.34, 'country_name': 'ARGENTINA'},
 {'total_balance': 247200.27000000002, 'country_name': 'BRAZIL'},
 {'total_balance': 284011.99, 'country_name': 'CANADA'},
 {'total_balance': 291863.05000000005, 'country_name': 'CHINA'},
 {'total_balance': 272480.14, 'country_name': 'EGYPT'},
 {'total_balance': 201760.96999999997, 'country_name': 'ETHIOPIA'},
 {'total_balance': 140663.19999999998, 'country_name': 'FRANCE'},
 {'total_balance': 243965.66000000006, 'country_name': 'GERMANY'},
 {'total_balance': 274001.8199999999, 'country_name': 'INDIA'},
 {'total_balance': 328113.13000000006, 'country_name': 'INDONESIA'},
 {'total_balance': 302886.37000000005, 'country_name': 'IRAN'},
 {'total_balance': 267563.05000000005, 'country_name': 'IRAQ'},
 {'total_balance': 332485.08, 'country_name': 'JAPAN'},
 {'total_balance': 226043.97999999992, 'country_name': 'JORDAN'},
 {'total_balance': 245055.49999999994, 'count

## Part 4: Close the database

Don't forget to close your database connection when you're done...

In [22]:
con.close()